# PRJDB36442 HUMAnN M vs S functional analysis

Scope: CHB gut shotgun cohort with histology severity groups (M vs S).
Primary goal: functional mechanisms associated with histology severity, not just taxa shifts.

## 1. Inputs

- Merged HUMAnN tables (pathways, coverage, gene families)
- Sample group mapping (M vs S)


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

BASE = Path('../..')  # notebook under output/jupyter-notebook
DATA_DIR = BASE / 'results' / 'processed' / 'PRJDB36442_humann'
GROUPS_PATH = BASE / 'results' / 'feasibility' / 'PRJDB36442_run_groups.tsv'

PATH_ABUND = DATA_DIR / 'merged_pathabundance.tsv.gz'
PATH_COV = DATA_DIR / 'merged_pathcoverage.tsv.gz'

for p in [PATH_ABUND, PATH_COV, GROUPS_PATH]:
    print(p, p.exists())


## 2. Load tables

We focus on unstratified pathway abundance for group-level functional shifts, and use stratified rows later for taxon-level attribution.

In [ ]:
def read_humann_table(path: Path):
    df = pd.read_csv(path, sep='	', header=0)
    df = df.rename(columns={df.columns[0]: 'feature'})
    return df


def normalize_sample_cols(df):
    new_cols = {}
    for c in df.columns:
        if c == 'feature':
            continue
        c2 = re.sub(r'_(Abundance|Coverage)(-RPKs)?$', '', c)
        new_cols[c] = c2
    return df.rename(columns=new_cols)

path_abund = normalize_sample_cols(read_humann_table(PATH_ABUND))
path_cov = normalize_sample_cols(read_humann_table(PATH_COV))

# split stratified vs unstratified
path_abund['is_stratified'] = path_abund['feature'].str.contains('\|')
path_cov['is_stratified'] = path_cov['feature'].str.contains('\|')

path_abund_unstrat = path_abund[~path_abund['is_stratified']].copy()
path_abund_strat = path_abund[path_abund['is_stratified']].copy()

path_cov_unstrat = path_cov[~path_cov['is_stratified']].copy()

path_abund_unstrat.head()


## 3. Load group mapping (M vs S)

In [ ]:
groups = pd.read_csv(GROUPS_PATH, sep='	')
# Expect columns: run_accession, group (M/S)
print(groups.head())
print(groups['group'].value_counts())

# Sample columns in tables are run accessions
samples = [c for c in path_abund_unstrat.columns if c not in ['feature','is_stratified']]
missing = set(samples) - set(groups['run_accession'])
assert not missing, f'Missing groups for: {sorted(missing)[:5]}'


## 4. Normalize pathway abundance (relative)

In [ ]:
abund = path_abund_unstrat.set_index('feature')
if 'is_stratified' in abund.columns:
    abund = abund.drop(columns=['is_stratified'])

# Drop UNMAPPED and UNINTEGRATED if desired for relative abundance analysis
abund = abund.drop(index=[i for i in ['UNMAPPED','UNINTEGRATED'] if i in abund.index])

# Relative abundance per sample
rel_abund = abund.div(abund.sum(axis=0), axis=1)
rel_abund.iloc[:5, :5]


## 5. Group-wise statistics (M vs S)

In [ ]:
from scipy.stats import mannwhitneyu

# map sample -> group
group_map = groups.set_index('run_accession')['group'].to_dict()

M_samples = [s for s in rel_abund.columns if group_map[s] == 'M']
S_samples = [s for s in rel_abund.columns if group_map[s] == 'S']

records = []
for feature, row in rel_abund.iterrows():
    m = row[M_samples].values
    s = row[S_samples].values
    # nonparametric test
    stat, p = mannwhitneyu(m, s, alternative='two-sided')
    # effect: difference in medians
    delta = np.median(s) - np.median(m)
    records.append((feature, np.median(m), np.median(s), delta, p))

res = pd.DataFrame(records, columns=['feature','median_M','median_S','delta_S_minus_M','p'])
res.head(10)


## 6. FDR correction (Benjamini–Hochberg)

In [ ]:
def bh_fdr(pvals):
    pvals = np.asarray(pvals, dtype=float)
    n = len(pvals)
    order = np.argsort(pvals)
    ranked = np.empty(n, dtype=float)
    prev = 1.0
    for i in range(n-1, -1, -1):
        rank = i + 1
        p = pvals[order[i]]
        q = min(prev, p * n / rank)
        prev = q
        ranked[order[i]] = q
    return ranked

res['q'] = bh_fdr(res['p'].values)
res = res.sort_values('q').reset_index(drop=True)
res.head(10)


## 7. Mechanism modules (MetaCyc; conservative + expanded)

In [ ]:
# Build a pathway catalog from unstratified pathways
path_catalog = pd.DataFrame({'feature': rel_abund.index})
path_catalog['name'] = path_catalog['feature'].str.split(':', n=1).str[1].fillna(path_catalog['feature']).str.strip()


def build_module_sets(path_catalog, rules, manual_include=None, manual_exclude=None):
    manual_include = manual_include or {}
    manual_exclude = manual_exclude or {}
    module_sets = {}
    for mod, patterns in rules.items():
        mask = pd.Series(False, index=path_catalog.index)
        for pat in patterns:
            mask = mask | path_catalog['name'].str.contains(pat, flags=re.IGNORECASE, regex=True)
        feats = path_catalog.loc[mask, 'feature'].tolist()
        feats += manual_include.get(mod, [])
        feats = [f for f in feats if f not in set(manual_exclude.get(mod, []))]
        module_sets[mod] = sorted(set(feats))
    return module_sets

# Conservative rules (primary analysis)
module_rules_conservative = {
    'bile_acids': [r'\bbile\b', r'cholate', r'chenodeoxycholate', r'deoxycholate', r'taurocholate', r'glycocholate'],
    'tryptophan_indole': [r'tryptophan', r'indole-3', r'kynurenine', r'anthranilate'],
    'lps_lipidA': [r'lipopolysaccharide', r'lipid A', r'kdo', r'O-antigen'],
    'scfa_butyrate': [r'butyrate', r'butanoate'],
    'scfa_propionate': [r'propionate'],
    'scfa_acetate': [r'acetate', r'acetyl-CoA'],
    'scfa_lactate_succinate': [r'lactate', r'succinate'],
}

# Expanded rules (sensitivity)
module_rules_expanded = {
    'bile_acids': module_rules_conservative['bile_acids'] + [r'bile acid', r'bile salt'],
    'tryptophan_indole': module_rules_conservative['tryptophan_indole'] + [r'indole', r'IAA'],
    'lps_lipidA': module_rules_conservative['lps_lipidA'] + [r'enterobacterial common antigen'],
    'scfa_butyrate': module_rules_conservative['scfa_butyrate'],
    'scfa_propionate': module_rules_conservative['scfa_propionate'] + [r'propan\b'],
    'scfa_acetate': module_rules_conservative['scfa_acetate'] + [r'acetyl coenzyme A'],
    'scfa_lactate_succinate': module_rules_conservative['scfa_lactate_succinate'] + [r'propionyl-CoA', r'succinyl-CoA'],
}

module_sets_cons = build_module_sets(path_catalog, module_rules_conservative)
module_sets_exp = build_module_sets(path_catalog, module_rules_expanded)

pd.DataFrame({
    'conservative': {k: len(v) for k,v in module_sets_cons.items()},
    'expanded': {k: len(v) for k,v in module_sets_exp.items()},
}).sort_index()


## 8. Module scores + FDR

In [ ]:
def module_scores_from_sets(rel_abund, module_sets):
    module_scores = {}
    for name, feats in module_sets.items():
        feats = [f for f in feats if f in rel_abund.index]
        if not feats:
            continue
        module_scores[name] = rel_abund.loc[feats].mean(axis=0)
    return pd.DataFrame(module_scores)

module_scores_cons = module_scores_from_sets(rel_abund, module_sets_cons)
module_scores_exp = module_scores_from_sets(rel_abund, module_sets_exp)

mod_records = []
for mod in module_scores_cons.columns:
    m = module_scores_cons.loc[M_samples, mod].values
    s = module_scores_cons.loc[S_samples, mod].values
    stat, p = mannwhitneyu(m, s, alternative='two-sided')
    delta = np.median(s) - np.median(m)
    mod_records.append((mod, np.median(m), np.median(s), delta, p))

mod_res_cons = pd.DataFrame(mod_records, columns=['module','median_M','median_S','delta_S_minus_M','p'])
mod_res_cons['q'] = bh_fdr(mod_res_cons['p'].values)
mod_res_cons = mod_res_cons.sort_values('q').reset_index(drop=True)

mod_records = []
for mod in module_scores_exp.columns:
    m = module_scores_exp.loc[M_samples, mod].values
    s = module_scores_exp.loc[S_samples, mod].values
    stat, p = mannwhitneyu(m, s, alternative='two-sided')
    delta = np.median(s) - np.median(m)
    mod_records.append((mod, np.median(m), np.median(s), delta, p))

mod_res_exp = pd.DataFrame(mod_records, columns=['module','median_M','median_S','delta_S_minus_M','p'])
mod_res_exp['q'] = bh_fdr(mod_res_exp['p'].values)
mod_res_exp = mod_res_exp.sort_values('q').reset_index(drop=True)

mod_res_cons


## 9. Stratified contribution (who drives a pathway)

In [ ]:
def stratified_contribution(pathway, strat_df, M_samples, S_samples, top_k=15):
    sub = strat_df[strat_df['feature'].str.startswith(pathway + '|')].copy()
    if sub.empty:
        return None
    sub = sub.set_index('feature')
    sub_rel = sub.div(sub.sum(axis=0), axis=1)
    m_mean = sub_rel[M_samples].mean(axis=1)
    s_mean = sub_rel[S_samples].mean(axis=1)
    out = pd.DataFrame({
        'feature': sub_rel.index,
        'mean_M': m_mean,
        'mean_S': s_mean,
        'delta_S_minus_M': s_mean - m_mean
    }).sort_values('delta_S_minus_M', ascending=False)
    return out.head(top_k)

contrib_tables = {}
for pathway in res['feature'].head(5):
    tbl = stratified_contribution(pathway, path_abund_strat, M_samples, S_samples, top_k=15)
    if tbl is not None:
        contrib_tables[pathway] = tbl

list(contrib_tables.keys())


## 10. Figures

In [ ]:
import matplotlib.pyplot as plt

FIGDIR = DATA_DIR / 'figures'
FIGDIR.mkdir(parents=True, exist_ok=True)

# colorblind-safe palette (Okabe-Ito)
COL_M = '#0072B2'
COL_S = '#D55E00'

# module boxplots
mods = list(module_scores_cons.columns)
if mods:
    data = []
    for mod in mods:
        for s in M_samples:
            data.append((mod, 'M', module_scores_cons.loc[s, mod]))
        for s in S_samples:
            data.append((mod, 'S', module_scores_cons.loc[s, mod]))
    df_plot = pd.DataFrame(data, columns=['module','group','value'])

    fig, ax = plt.subplots(figsize=(7.5, 4.2))
    positions = np.arange(len(mods))
    width = 0.32

    for i, mod in enumerate(mods):
        m_vals = df_plot[(df_plot['module']==mod) & (df_plot['group']=='M')]['value'].values
        s_vals = df_plot[(df_plot['module']==mod) & (df_plot['group']=='S')]['value'].values
        bp = ax.boxplot([m_vals, s_vals], positions=[i - width/2, i + width/2], widths=0.25,
                        patch_artist=True, showfliers=False)
        bp['boxes'][0].set(facecolor=COL_M, alpha=0.5)
        bp['boxes'][1].set(facecolor=COL_S, alpha=0.5)
        for median in bp['medians']:
            median.set(color='black', linewidth=1.2)

        rng = np.random.default_rng(42 + i)
        ax.scatter(rng.normal(i - width/2, 0.04, size=len(m_vals)), m_vals, s=18, color=COL_M, alpha=0.7, linewidths=0)
        ax.scatter(rng.normal(i + width/2, 0.04, size=len(s_vals)), s_vals, s=18, color=COL_S, alpha=0.7, linewidths=0)

    ax.set_xticks(positions)
    ax.set_xticklabels(mods, rotation=20, ha='right')
    ax.set_ylabel('Relative abundance (module score)')
    ax.set_title('Module scores by histology group (M vs S)')
    ax.legend(handles=[
        plt.Line2D([0],[0], marker='o', color='w', label='M', markerfacecolor=COL_M, markersize=7),
        plt.Line2D([0],[0], marker='o', color='w', label='S', markerfacecolor=COL_S, markersize=7)
    ], frameon=False, loc='upper right')
    fig.tight_layout()
    fig.savefig(FIGDIR / 'module_scores_MvS.png', dpi=300)
    fig.savefig(FIGDIR / 'module_scores_MvS.pdf')
    plt.close(fig)

# top pathways lollipop
res_top = res.head(20).copy()
res_top['label'] = res_top['feature'].str.split(':', n=1).str[1].fillna(res_top['feature']).str.strip()
res_top['label'] = res_top['label'].str.slice(0, 50)
res_top = res_top.sort_values('delta_S_minus_M')

fig, ax = plt.subplots(figsize=(7.5, 6.0))
colors = ['#D55E00' if x>0 else '#0072B2' for x in res_top['delta_S_minus_M']]
ax.hlines(y=res_top['label'], xmin=0, xmax=res_top['delta_S_minus_M'], color=colors, alpha=0.7)
ax.scatter(res_top['delta_S_minus_M'], res_top['label'], color=colors, s=30)
ax.axvline(0, color='gray', linewidth=0.8)
ax.set_xlabel('Delta (S - M)')
ax.set_ylabel('Pathway')
ax.set_title('Top 20 pathways by FDR')
fig.tight_layout()
fig.savefig(FIGDIR / 'pathways_top20_lollipop.png', dpi=300)
fig.savefig(FIGDIR / 'pathways_top20_lollipop.pdf')
plt.close(fig)


## 11. Output tables for reporting

In [ ]:
OUTDIR = DATA_DIR
OUTDIR.mkdir(parents=True, exist_ok=True)

res.to_csv(OUTDIR / 'pathway_M_vs_S_stats.tsv', sep='	', index=False)
mod_res_cons.to_csv(OUTDIR / 'module_M_vs_S_stats_conservative.tsv', sep='	', index=False)
mod_res_exp.to_csv(OUTDIR / 'module_M_vs_S_stats_expanded.tsv', sep='	', index=False)

# Save module definitions for audit
with (OUTDIR / 'module_sets_conservative.tsv').open('w') as f:
    f.write('module\tfeature\n')
    for mod, feats in module_sets_cons.items():
        for feat in feats:
            f.write(f"{mod}\t{feat}\n")

with (OUTDIR / 'module_sets_expanded.tsv').open('w') as f:
    f.write('module\tfeature\n')
    for mod, feats in module_sets_exp.items():
        for feat in feats:
            f.write(f"{mod}\t{feat}\n")

# Save stratified contributions
for pathway, tbl in contrib_tables.items():
    safe = pathway.split(':')[0]
    tbl.to_csv(OUTDIR / f'strat_contrib_{safe}.tsv', sep='	', index=False)

print('Saved:', OUTDIR / 'pathway_M_vs_S_stats.tsv')
